In [ ]:
using Plots, RebindingKineticsMLE, StatsBase

# Introduction

This is a minimal example on how to apply our maximum likelihood estimator to a set of pulling traces.  To get a feeling for the quality of the fit, we compare our results to rate maps generated via the method by Zhang & Dudko [https://doi.org/10.1073/pnas.1309101110].  

For more details on the theoretical framework, please refer to the associated publication:
> J. T. Bullerjahn and G. Hummer, "Rebinding kinetics from single-molecule force spectroscopy experiments close to equilibrium", ...

# Generate synthetic data

Each pulling trace (<code>Array{Float64,2}</code>) is defined by the applied force and the state the system occupies at every measurement.  A collection of pulling traces should therefore be of the type <code>Array{Array{Float64,2},1}</code>.  

Here, we use the Gillespie stochastic simulation algorithm to generate synthetic data for a linear force protocol $F(t) \propto \dot{F} t$.  The data set is made up of $M$ pulling traces of varying lengths.  

In [ ]:
include("./linear_gillespie_simulation_functions.jl")

const Δt = 1e-7 # time step with dimension [Δt] = s

const M = 10 # Number of traces
const dF = 100.0 # Loading rate
const F_0 = 6.0 # Force offset between states with dimension [F_0] = pN

const Δx_off = 0.4
const Δx_on = 0.2
const k_off0 = 3.0
const k_on0 = 4000.0
ground_truth = [Δx_off, Δx_on, k_off0, k_on0]

data = make_data(M,dF,F_0,Δt,ground_truth)
b_to_b, u_to_b, b_to_u, u_to_u = sort_data(data);

# Analyzing the data

## Parameter estimation

In [ ]:
MLE_estimates, MLE_errors = MLE_estimator(data,Δt)

println("Estimates:")
println(string("Δx_off = ", MLE_estimates[1], " ± ", MLE_errors[1]))
println(string("Δx_on = ", MLE_estimates[2], " ± ", MLE_errors[2]))
println(string("k_off0 = ", MLE_estimates[3], " ± ", MLE_errors[3]))
println(string("k_on0 = ", MLE_estimates[4], " ± ", MLE_errors[4]))

println()

println("Ground truth:")
println(string("Δx_off = ", Δx_off))
println(string("Δx_on = ", Δx_on))
println(string("k_off0 = ", k_off0))
println(string("k_on0 = ", k_on0))

## Compare to rate maps

In [ ]:
include("./rate_map_functions.jl")

N_bin = 30
off_rate, on_rate = rate_map(b_to_u,u_to_b,dF,F_0,M,N_bin)

exact_off_rate = k_off.(ground_truth[1],ground_truth[3],off_rate[:,1])
exact_on_rate = k_on.(ground_truth[2],ground_truth[4],on_rate[:,1],0.0)

MLE_off_rate = k_off.(MLE_estimates[1],MLE_estimates[3],off_rate[:,1])
MLE_on_rate = k_on.(MLE_estimates[2],MLE_estimates[4],on_rate[:,1],0.0)

p1 = plot(off_rate[:,1], off_rate[:,2], seriestype = :scatter, yerror=off_rate[:,3], yscale=:log10, ylims=(1,10000), label="off-rate map")
p2 = plot!(p1, on_rate[:,1], on_rate[:,2], seriestype = :scatter, yerror=on_rate[:,3], yscale=:log10, label="on-rate map")
p3 = plot!(p2, off_rate[:,1], hcat([exact_off_rate, MLE_off_rate]...), yscale=:log10, labels=["exact off-rate" "MLE off-rate"])
p4 = plot!(p3, on_rate[:,1], hcat([exact_on_rate, MLE_on_rate]...), yscale=:log10, labels=["exact on-rate" "MLE on-rate"])